In [ ]:
import cudf
import cuml
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
X = cudf.read_parquet("../data/processed/train.parquet")

In [ ]:
X.head()

In [ ]:
y = cudf.read_csv("../data/raw/train_gt.csv", index_col=0, skiprows=0)

In [ ]:
df = X.drop(columns="target").join(y)
df.head()

In [ ]:
df = df.dropna()
X = df.drop(columns="5408")

In [ ]:
from cuml.preprocessing import Normalizer

X = Normalizer().fit_transform(X)

In [ ]:
from cuml.manifold.umap import UMAP as cuUMAP
from umap import UMAP

trained_UMAP = cuUMAP(n_neighbors=8).fit(X)
mod = UMAP(n_neighbors=8).fit(X.to_pandas())
X_embedded = trained_UMAP.transform(X)

cu_score = cuml.metrics.trustworthiness(X, X_embedded)

print(" cuml's trustworthiness score : ", cu_score)

In [ ]:
X_emb = X_embedded.to_numpy()
df_np = df.to_pandas()

In [ ]:
# plot the umap embedding

plt.figure(figsize=(10, 10))
sns.scatterplot(x=X_emb[:, 0], y=X_emb[:, 1], hue=df_np["5408"])

In [ ]:
import umap.plot

In [ ]:
umap.plot.points(
    mod,
    labels=df_np["5408"],
    width=1200,
    height=800,
    color_key={"active": "red", "inactive": "#8dd3c7"},
)
plt.title(f"UMAP 2D Embedding on L2-normalized Data (T={cu_score:.2f})")
plt.savefig("../output/umap_2d.png", bbox_inches="tight")

In [ ]:
X.kurtosis().to_pandas().plot(kind="bar", figsize=(20, 10))